In [ ]:
from collections import namedtuple
import pathlib
import pprint
import random

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

In [ ]:
from orchid import (core as oc,
                    net_quantity as opq,
                    measurement as om,
                    project as op,
                    project_loader as opl,
                    reference_origins as origin,
                    unit_system as units)

In [ ]:
import toolz.curried as toolz

In [ ]:
# noinspection PyUnresolvedReferences
from Orchid.FractureDiagnostics import WellReferenceFrameXy, DepthDatum
# noinspection PyUnresolvedReferences
import UnitsNet
# noinspection PyUnresolvedReferences
from System import Array, Int32

In [ ]:
project_filenames = {
    'bakken': 'frankNstein_Bakken_UTM13_FEET.ifrac',
    'montney': 'Project-frankNstein_Montney_UTM13_METERS.ifrac',
    'permian': 'Project_frankNstein_Permian_UTM13_FEET.ifrac'
}

In [ ]:
projects = toolz.pipe(
    project_filenames,
    toolz.valmap(lambda fn: str(pathlib.Path(r'c:\src\Orchid.IntegrationTestData').joinpath(fn))),
    toolz.valmap(toolz.curry(oc.load_project)),
)
pprint.pprint(projects)

In [ ]:
native_projects = toolz.valmap(lambda p: p.dom_object, projects)
pprint.pprint(native_projects)

In [ ]:
wells = toolz.pipe(native_projects,
    toolz.valmap(lambda native_project: native_project.Wells.Items),
    toolz.valmap(lambda well_map: [(w.Name, w) for w in well_map]),
    toolz.valmap(dict))
pprint.pprint(wells)

In [ ]:
trajectories = toolz.valmap(toolz.valmap(lambda w: w.Trajectory), wells)
pprint.pprint(trajectories)

In [ ]:
eastings = toolz.valmap(
    toolz.valmap(
        lambda t: t.GetEastingArray(WellReferenceFrameXy.Project),
        toolz.map(lambda e: e.Value),
        
        lambda t: np.fromiter(t.GetEastingArray(WellReferenceFrameXy.Project), np.dtype(float))
    ), 
    trajectories
)
pprint.pprint(eastings)

In [ ]:
bakken_wells = project_wells['bakken']

In [ ]:
bakken_demo_1h = toolz.get_in(['bakken', 'Demo_1H'], project_wells)

In [ ]:
trajectory = bakken_demo_1h.Trajectory

In [ ]:
eastings = trajectory.GetEastingArray(WellReferenceFrameXy.Project)

In [ ]:
len(eastings)

In [ ]:
toolz.pipe(eastings,
           toolz.take(5),
           toolz.map(str),
           list)

In [ ]:
np_eastings = toolz.pipe(
    eastings,
    toolz.map(opq.as_measurement(units.UsOilfield.LENGTH)),
    toolz.map(lambda m: m.magnitude),
    list,
    np.array,
)

In [ ]:
measured_eastings = om.registry.Quantity(np_eastings, om.registry.ft)

In [ ]:
measured_eastings[:5]